In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model và tokenizer
checkpoint = "mr4/phobert-base-vi-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

def predict_sentiment_score(text, max_length=256, stride=50):
    """
    Dự đoán điểm cảm xúc của văn bản dài bằng PhoBERT.

    Args:
        text (str): Văn bản cần phân tích.
        max_length (int): Số token tối đa của mô hình.
        stride (int): Mức độ chồng lấn giữa các đoạn.

    Returns:
        dict: Xác suất của các nhãn cảm xúc {label: probability}
    """
    # Tokenize văn bản dài, cắt thành từng đoạn
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True
    )

    input_keys = ["input_ids", "attention_mask"]
    all_predictions = []

    with torch.no_grad():  # Không tính gradient để tăng tốc
        for i in range(len(inputs["input_ids"])):  # Duyệt qua từng đoạn đã cắt
            segment = {k: inputs[k][i].unsqueeze(0) for k in input_keys}
            outputs = model(**segment)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            all_predictions.append(predictions[0].tolist())

    # Tính trung bình dự đoán từ tất cả các đoạn
    avg_predictions = torch.tensor(all_predictions).mean(dim=0)

    # Chuyển kết quả thành dict với nhãn và xác suất tương ứng
    sentiment_scores = {
        model.config.id2label[j]: avg_predictions[j].item()
        for j in range(len(avg_predictions))
    }

    return sentiment_scores


In [4]:
text = """" Trong một cuộc họp báo diễn ra vào sáng nay (22/3), ông Nguyễn Đức Chung, Chủ tịch UBND TP Hà Nội, cho biết đã đề xuất Bộ Y tế cấp 1 triệu liều vaccine ngừa COVID-19 cho Hà Nội."""
predict_sentiment_score(text)

{'Tiêu cực': 0.047979678958654404,
 'Tích cực': 0.12986205518245697,
 'Trung tính': 0.8221582174301147}

In [ ]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer
from underthesea import word_tokenize  # Dùng để word-segmented văn bản

# Load mô hình PhoBERT đã fine-tuned cho phân tích cảm xúc
checkpoint = "wonrax/phobert-base-vietnamese-sentiment"
model = RobertaForSequenceClassification.from_pretrained(checkpoint)

# Load tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=False)

def predict_sentiment_score_wonrax(text, max_length=256, stride=50):
    """
    Dự đoán điểm cảm xúc của văn bản dài bằng PhoBERT.

    Args:
        text (str): Văn bản cần phân tích.
        max_length (int): Số token tối đa của mô hình.
        stride (int): Mức độ chồng lấn giữa các đoạn.

    Returns:
        dict: Xác suất của các nhãn cảm xúc {label: probability}
    """
    # Bước 1: Tách từ bằng Underthesea để phù hợp với PhoBERT
    word_segmented_text = word_tokenize(text, format="text")

    # Bước 2: Tokenize văn bản dài, cắt thành từng đoạn nhỏ
    inputs = tokenizer(
        word_segmented_text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True
    )

    input_keys = ["input_ids", "attention_mask"]
    all_predictions = []

    # Bước 3: Chạy mô hình trên từng đoạn nhỏ
    with torch.no_grad():  # Không tính gradient để tăng tốc
        for i in range(len(inputs["input_ids"])):  # Duyệt qua từng đoạn đã cắt
            segment = {k: inputs[k][i].unsqueeze(0) for k in input_keys}
            outputs = model(**segment)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            all_predictions.append(predictions[0].tolist())

    # Bước 4: Tính trung bình dự đoán từ tất cả các đoạn
    avg_predictions = torch.tensor(all_predictions).mean(dim=0)

    # Bước 5: Chuyển kết quả thành dict với nhãn và xác suất tương ứng
    sentiment_scores = {
        model.config.id2label[j]: avg_predictions[j].item()
        for j in range(len(avg_predictions))
    }

    return sentiment_scores




c:\Users\LAP\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
c:\Users\LAP\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Sentiment Scores: {'NEG': 0.6783797740936279, 'POS': 0.03750285878777504, 'NEU': 0.2841174304485321}


In [8]:
# Test thử với văn bản dài
text = """Thông tin Tổng thống Mỹ Donald Trump đưa ra lệnh áp thuế đối ứng với Việt Nam lên tới 46% làm thị trường chứng khoán Việt Nam chao đảo.
"""
result = predict_sentiment_score_wonrax(text)

# In kết quả
print("Sentiment Scores:", result)

Sentiment Scores: {'NEG': 0.868488073348999, 'POS': 0.02748456411063671, 'NEU': 0.10402730852365494}


In [6]:
import pandas as pd
df = pd.read_csv("filtered_cafef_news_details.csv")

In [3]:
text_1 = """Mỹ đánh thuế 46% với Việt Nam, cao nhất trong số các nước bị áp thuế.""" 
text_2 = "VN-Index tăng 100 điểm."  
text_3 = "VN-Index tăng 500 điểm."  

print(predict_sentiment_score(text_1))  
print(predict_sentiment_score(text_2))  
print(predict_sentiment_score(text_3))  


NameError: name 'predict_sentiment_score' is not defined

In [8]:
import re

text = """TIN MỚIThị trường diễn ra giằng co và lình xình quanh tham chiếu trong suốt phiên giao dịch. Cả bên bán và bên mua đều dè chừng khi VN-Index có chuỗi phiên tăng liên tục và đang ở trên vùng giá cao. Bất ngờ xảy ra vào cuối phiên khi bên bán xả hàng mạnh, khiến đà bán lan rộng ra các nhóm ngành.Cổ phiếu    dầu khí trở thành nhóm giảm mạnh nhất khi BSR, PVS, PVD, PLX, PVB... đều nhuốm đỏ. Nhóm cổ phiếu đầu tư công bốc hơi đáng kể khi, VCG giảm 5,4%, HHV giảm 3,5%, LCG giảm 3,7%, CTD giảm 4,6%, HCB giảm 4,2%, FCN giảm 3,8%...Áp lực chốt lời cũng diễn ra ở nhóm     cổ phiếu       ngân hàng    khi MBB giảm 1,2%, TCB giảm 2%, VPB giảm 1,8%, HDB giảm 2,1%, MSB giảm 2,1%, VCB giảm 1,8%. Những mã cổ phiếu ngân hàng này cũng tạo sức ép lớn nhất đến chỉ số VN-Index hôm nay.Cổ phiếu     bất động sản    có sự phân hóa. Dù số mã giảm điểm gấp 3 lần số mã tăng nhưng nhờ sự gánh đỡ từ cổ phiếu 'nhà"     Vingroup    , về tổng thể nhóm cổ phiếu này vẫn tăng điểm. Trong đó, VRE tăng 2,7%, VHM tăng 1,8%, đặc biệtVICtăng trần lên 51.400 đồng/cổ phiếu, mức cao nhất trong vòng 18 tháng trở lại đây. Chỉ riêng VIC đã đóng góp tới hơn 3 điểm vào chỉ số VN-Index giúp thị trường bớt đà giảm.Sắc đỏ lan rộng trong phiên hôm nay (13/3).Kết thúc phiên giao dịch, VN-Index giảm 8,14 điểm về 1.326,2 điểm; HNX-Index giảm 0,56 điểm còn 241,3 điểm, Upcom giảm 0,43 điểm, còn 98,8 điểm. Độ rộng thị trường nghiêng hoàn toàn về sắc đỏ với 303 mã giảm, và 91 mã tăng. Số mã giảm cũng chiếm đáng kể bên rổ VN30 với 19 mã, so với 9 mã tăng và 2 mã đi ngang.Thanh khoản trong phiên hôm nay được cải thiện với hơn 1 tỷ cổ phiếu được trao tay, tương đương giá trị giao dịch đạt gần 24.500 tỷ đồng.Đáng chú ý, khi khối nội xả hàng, khối ngoại trong phiên hôm nay lại mua khá tích cực. Khối này mua vào lượng cổ phiếu trị giá 2.522 tỷ đồng và bán ra 2.370 tỷ đồng trên sàn HoSE.Theo thống kê, 2 cổ phiếu được nhà đầu tư ngoại gom mạnh là VJC và SSI Phía ngược lại, khối ngoại bán ròng hơn 100 tỷ đồng 2 mã VCB và VNM."""

In [9]:
# Danh sách từ khóa cần lọc
keywords = ["dầu khí",'PLX']

# Tách câu theo dấu chấm, đảm bảo không làm vỡ số thập phân
sentences = re.split(r'(?<!\d)\.(?!\d)', text)

# Loại bỏ khoảng trắng thừa và câu rỗng
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

# Lọc câu chứa bất kỳ từ khóa nào trong danh sách
filtered_sentences = [sentence for sentence in sentences if any(keyword.lower() in sentence.lower() for keyword in keywords)]
# Nối các câu thành một đoạn văn mới
filtered_text = ". ".join(filtered_sentences) + "."

# In kết quả
print("🔎 Đoạn văn mới chứa các câu lọc theo từ khóa:")
print(filtered_text)
# In kết quả
print("🔎 Các câu chứa từ khóa:")
for i, sentence in enumerate(filtered_sentences, 1):
    print(f"- {sentence}")

🔎 Đoạn văn mới chứa các câu lọc theo từ khóa:
Cổ phiếu    dầu khí trở thành nhóm giảm mạnh nhất khi BSR, PVS, PVD, PLX, PVB.
🔎 Các câu chứa từ khóa:
- Cổ phiếu    dầu khí trở thành nhóm giảm mạnh nhất khi BSR, PVS, PVD, PLX, PVB


In [17]:
import pandas as pd
import re
df = pd.read_csv("cafef_news_details.csv")
# Danh sách từ khóa cần lọc
keywords = ["opec"]

# Hàm xử lý lọc câu chứa từ khóa
def filter_sentences(text, keywords):
    if pd.isna(text):  # Kiểm tra nếu giá trị NaN
        return ""
    
    # Tách câu theo dấu chấm, giữ nguyên số thập phân
    sentences = re.split(r'(?<!\d)\.(?!\d)', text)
    
    # Loại bỏ khoảng trắng và câu rỗng
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Lọc các câu chứa từ khóa
    filtered_sentences = [sentence for sentence in sentences if any(keyword.lower() in sentence.lower() for keyword in keywords)]

    # Nối lại thành đoạn văn bản
    return ". ".join(filtered_sentences) + "." if filtered_sentences else ""

# Áp dụng hàm vào cột mới
df["filtered_content"] = df["content"].apply(lambda x: filter_sentences(x, keywords))



In [18]:
# Hiển thị kết quả
df[["content", "filtered_content"]].to_csv("test_processing.csv", index=False)

In [19]:
df_fil = pd.read_csv("test_processing.csv")
# Lọc các dòng có giá trị trong cột "filtered_content" (loại bỏ NaN)
df_fil_filtered = df_fil.dropna(subset=["filtered_content"])

df_fil_filtered.to_csv("test_processing.csv", index=False)